In [1]:
import os
import time
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from stable_baselines3 import PPO, DQN
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
from scipy.optimize import differential_evolution

In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Force NVIDIA GPU (GPU 0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Verify GPU selection
print(f"Using device: {device}")
print("GPU Name:", torch.cuda.get_device_name(device))

# Create a random tensor and move it to the selected GPU
x = torch.randn(1000, 1000).to(device)
print("✅ Tensor successfully moved to:", x.device)

CUDA Available: True
GPU Count: 1
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
Using device: cuda
Using device: cuda:0
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
✅ Tensor successfully moved to: cuda:0


In [4]:
# Directories
data_dir = "E:/Works/12. Plant Diseases Classification/Datasets/Work_Dataset"
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")
test_dir = os.path.join(data_dir, "test")
external_test_dir = os.path.join(data_dir, "External_testing_dataset")

In [5]:
# Hyperparameters
batch_size = 16
num_epochs = 50
learning_rate = 0.001
img_size = 224
weight_decay = 1e-4
patience = 5

In [6]:
# Transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
# Load Data
train_dataset = ImageFolder(train_dir, transform=train_transforms)
valid_dataset = ImageFolder(valid_dir, transform=valid_test_transforms)
test_dataset = ImageFolder(test_dir, transform=valid_test_transforms)
external_test_dataset = ImageFolder(external_test_dir, transform=valid_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
external_test_loader = DataLoader(external_test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# Class Names
class_names = train_dataset.classes
num_classes = len(class_names)
print(f"Classes: {class_names}")

Classes: ['A_blk_rot_f', 'A_blk_spot', 'A_e_canker', 'A_gl_lf_spot', 'A_healthy_f', 'A_healthy_l', 'A_m_virus', 'Av_alg_lf_spot', 'Av_br_canker', 'Av_healthy_l', 'G_blk_spot_c', 'G_healthy_c', 'Kf_bac_canker', 'Kf_healthy_l', 'P_canker', 'P_fr_blight', 'P_healthy_f', 'P_healthy_l', 'P_s_pit', 'P_scab']


In [9]:
# Compute Class Weights
labels = np.array(train_dataset.targets)
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

In [10]:
# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction="none", weight=self.alpha)
        pt = torch.exp(-ce_loss)  # Probabilities of correct class
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss  # Focal loss formula

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [11]:
# Temperature Scaling Function
class TemperatureScaling(nn.Module):
    def __init__(self, init_temp=1.5):
        super(TemperatureScaling, self).__init__()
        self.temperature = nn.Parameter(torch.tensor(init_temp))

    def forward(self, logits):
        return logits / self.temperature

In [12]:
# Define NTSO Feature Selector
class NTSO_FeatureSelector:
    def __init__(self, num_features, num_tunas=30, max_iter=50, alpha=0.6, beta=0.4):
        self.num_features = num_features
        self.num_tunas = num_tunas  # Population size (tuna swarm)
        self.max_iter = max_iter  # Number of iterations
        self.alpha = alpha  # Exploration weight
        self.beta = beta  # Exploitation weight

    def fitness_function(self, features):
        """
        Placeholder fitness function.
        Ideally, this should evaluate the feature subset based on model performance.
        """
        return np.random.rand()  # Replace with actual evaluation metric

    def select_features(self, train_loader, valid_loader):
        print("Selecting features using Novel Tuna Swarm Optimization (NTSO)...")

        # Initialize tuna positions (binary feature selection)
        tuna_swarm = np.random.randint(0, 2, (self.num_tunas, self.num_features))
        fitness_values = np.array([self.fitness_function(tuna) for tuna in tuna_swarm])

        # Identify the best tuna (leader)
        best_idx = np.argmin(fitness_values)
        leader = tuna_swarm[best_idx]

        # Optimization loop
        for iteration in range(self.max_iter):
            for i in range(self.num_tunas):
                if i == best_idx:
                    continue  # Skip the leader

                # Tuna movement (balancing exploration & exploitation)
                random_exploration = self.alpha * np.random.uniform(-1, 1, self.num_features)
                guided_exploitation = self.beta * (leader - tuna_swarm[i])

                # Update tuna position
                new_position = tuna_swarm[i] + random_exploration + guided_exploitation
                new_position = np.clip(new_position, 0, 1)

                # Convert to binary (0 or 1)
                new_position = (np.random.rand(self.num_features) < new_position).astype(int)

                # Evaluate new position
                new_fitness = self.fitness_function(new_position)
                if new_fitness < fitness_values[i]:  # Minimization problem
                    tuna_swarm[i] = new_position
                    fitness_values[i] = new_fitness

            # Update leader if a better solution is found
            best_idx = np.argmin(fitness_values)
            leader = tuna_swarm[best_idx]
            best_fitness = fitness_values[best_idx]

            print(f"Iteration {iteration+1}/{self.max_iter} - Best Fitness: {best_fitness:.4f}")

        # Select features from the best solution
        selected_features = np.where(leader > 0.5)[0].tolist()
        print(f"Selected {len(selected_features)} features out of {self.num_features}")

        return selected_features

In [13]:
# Define PMOA Feature Selector
class PMOA_FeatureSelector:
    def __init__(self, num_features, num_agents=30, max_iter=50, prob_explore=0.7):
        self.num_features = num_features
        self.num_agents = num_agents  # Population size (agents)
        self.max_iter = max_iter  # Number of iterations
        self.prob_explore = prob_explore  # Probability of exploration

    def fitness_function(self, features):
        """
        Placeholder fitness function.
        Ideally, this should evaluate the feature subset based on model performance.
        """
        return np.random.rand()  # Replace with actual evaluation metric

    def select_features(self, train_loader, valid_loader):
        print("Selecting features using Probabilistic Meta-Heuristic Optimization Algorithm (PMOA)...")

        # Initialize agent positions (binary feature selection)
        agents = np.random.randint(0, 2, (self.num_agents, self.num_features))
        fitness_values = np.array([self.fitness_function(agent) for agent in agents])

        # Identify the best agent (leader)
        best_idx = np.argmin(fitness_values)
        leader = agents[best_idx]

        # Optimization loop
        for iteration in range(self.max_iter):
            for i in range(self.num_agents):
                if i == best_idx:
                    continue  # Skip the leader

                # Probabilistic decision: exploration or exploitation
                if np.random.rand() < self.prob_explore:
                    # Exploration: Randomly perturb agent's position
                    random_exploration = np.random.randint(0, 2, self.num_features)
                    new_position = np.bitwise_xor(agents[i], random_exploration)
                else:
                    # Exploitation: Move towards the leader
                    new_position = (leader + agents[i]) // 2  # Average movement

                # Convert to binary (0 or 1)
                new_position = (np.random.rand(self.num_features) < new_position).astype(int)

                # Evaluate new position
                new_fitness = self.fitness_function(new_position)
                if new_fitness < fitness_values[i]:  # Minimization problem
                    agents[i] = new_position
                    fitness_values[i] = new_fitness

            # Update leader if a better solution is found
            best_idx = np.argmin(fitness_values)
            leader = agents[best_idx]
            best_fitness = fitness_values[best_idx]

            print(f"Iteration {iteration+1}/{self.max_iter} - Best Fitness: {best_fitness:.4f}")

        # Select features from the best solution
        selected_features = np.where(leader > 0.5)[0].tolist()
        print(f"Selected {len(selected_features)} features out of {self.num_features}")

        return selected_features

In [14]:
# Define LOA Feature Selector
class LOA_FeatureSelector:
    def __init__(self, num_features, num_lemurs=30, max_iter=50, alpha=0.8, beta=0.2):
        self.num_features = num_features
        self.num_lemurs = num_lemurs  # Population size
        self.max_iter = max_iter  # Number of iterations
        self.alpha = alpha  # Exploration factor
        self.beta = beta  # Exploitation factor

    def fitness_function(self, features):
        """
        Placeholder fitness function.
        Ideally, this should evaluate the feature subset based on model performance.
        """
        return np.random.rand()  # Replace with actual evaluation metric

    def select_features(self, train_loader, valid_loader):
        print("Selecting features using Lemurs Optimization Algorithm (LOA)...")

        # Initialize lemur positions (binary feature selection)
        lemurs = np.random.randint(0, 2, (self.num_lemurs, self.num_features))  
        fitness_values = np.array([self.fitness_function(lemur) for lemur in lemurs])

        # Identify best (leader) lemur
        best_idx = np.argmin(fitness_values)
        leader = lemurs[best_idx]

        # Optimization loop
        for iteration in range(self.max_iter):
            for i in range(self.num_lemurs):
                if i == best_idx:
                    continue  # Skip the leader

                # Update position based on leader
                rand_factor = np.random.uniform(0, 1, self.num_features)
                new_position = (
                    leader * self.alpha + lemurs[i] * self.beta + rand_factor * (1 - self.beta)
                )
                new_position = np.clip(new_position, 0, 1)  
                new_position = (np.random.rand(self.num_features) < new_position).astype(int)  

                # Evaluate new position
                new_fitness = self.fitness_function(new_position)
                if new_fitness < fitness_values[i]:  # Minimization problem
                    lemurs[i] = new_position
                    fitness_values[i] = new_fitness

            # Update leader if a better solution is found
            best_idx = np.argmin(fitness_values)
            leader = lemurs[best_idx]
            best_fitness = fitness_values[best_idx]

            print(f"Iteration {iteration+1}/{self.max_iter} - Best Fitness: {best_fitness:.4f}")

        # Select features from the best solution
        selected_features = np.where(leader > 0.5)[0].tolist()
        print(f"Selected {len(selected_features)} features out of {self.num_features}")

        return selected_features

In [15]:
# Define OOA Feature Selector
class OOA_FeatureSelector:
    def __init__(self, num_features, num_osp=30, max_iter=50, dive_factor=0.5):
        self.num_features = num_features
        self.num_osp = num_osp  # Population size (ospreys)
        self.max_iter = max_iter  # Number of iterations
        self.dive_factor = dive_factor  # Controls exploration intensity

    def fitness_function(self, features):
        """
        Placeholder fitness function.
        Ideally, this should evaluate the feature subset based on model performance.
        """
        return np.random.rand()  # Replace with actual evaluation metric

    def select_features(self, train_loader, valid_loader):
        print("Selecting features using Osprey Optimization Algorithm (OOA)...")

        # Initialize osprey positions (binary feature selection)
        ospreys = np.random.randint(0, 2, (self.num_osp, self.num_features))  
        fitness_values = np.array([self.fitness_function(osp) for osp in ospreys])

        # Identify the best (leader) osprey
        best_idx = np.argmin(fitness_values)
        leader = ospreys[best_idx]

        # Optimization loop
        for iteration in range(self.max_iter):
            for i in range(self.num_osp):
                if i == best_idx:
                    continue  # Skip the leader

                # Osprey diving behavior (exploration)
                dive_strength = self.dive_factor * (1 - iteration / self.max_iter)
                random_dive = np.random.uniform(-dive_strength, dive_strength, self.num_features)
                new_position = leader + random_dive  

                # Convert to binary (0 or 1)
                new_position = np.clip(new_position, 0, 1)
                new_position = (np.random.rand(self.num_features) < new_position).astype(int)

                # Evaluate new position
                new_fitness = self.fitness_function(new_position)
                if new_fitness < fitness_values[i]:  # Minimization problem
                    ospreys[i] = new_position
                    fitness_values[i] = new_fitness

            # Update leader if a better solution is found
            best_idx = np.argmin(fitness_values)
            leader = ospreys[best_idx]
            best_fitness = fitness_values[best_idx]

            print(f"Iteration {iteration+1}/{self.max_iter} - Best Fitness: {best_fitness:.4f}")

        # Select features from the best solution
        selected_features = np.where(leader > 0.5)[0].tolist()
        print(f"Selected {len(selected_features)} features out of {self.num_features}")

        return selected_features

In [16]:
# Define the number of features
num_features = 1000

# Metaheuristic Algorithms with correct parameter passing
metaheuristics = {   
    "NTSO": NTSO_FeatureSelector(num_features),  
    "PMOA": PMOA_FeatureSelector(num_features),  
    "LOA": LOA_FeatureSelector(num_features),   
    "OOA": OOA_FeatureSelector(num_features)
}

In [17]:
# Model Selection
models = {
    "ConvNeXt-Tiny": torchvision.models.convnext_tiny(weights=torchvision.models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1),
    "EfficientNetV2": torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1"),
    "RegNetY-8GF": torchvision.models.regnet_y_8gf(weights="IMAGENET1K_V1"),
    "MaxViT": torchvision.models.maxvit_t(weights="IMAGENET1K_V1"),
}

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [18]:
# Modify models for classification
for name, model in models.items():
    if "convnext" in name.lower():
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
    elif "efficientnet" in name.lower():
        in_features = model.classifier[-1].in_features  
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    elif "regnet" in name.lower():
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif "maxvit" in name.lower():
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)

    # Add Temperature Scaling
    model.temperature_scaling = TemperatureScaling()
    
    model.to(device)

In [19]:
# Loss Function: Choose between Class Weighted Loss or Focal Loss
use_focal_loss = True  # Set to False to use standard class-weighted cross-entropy

if use_focal_loss:
    criterion = FocalLoss(alpha=class_weights)
else:
    criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizers with Weight Decay
optimizers = {name: optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 
              for name, model in models.items()}

In [20]:
# Train Function
def train_and_evaluate(model, optimizer, train_loader, valid_loader, num_epochs, device):
    history = {"train_loss": [], "train_acc": [], "valid_loss": [], "valid_acc": [], "f1": [], 
               "precision": [], "recall": [], "auc": [], "time": 0}
    start_time = time.time()
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    best_val_loss = float("inf")
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            outputs = model.temperature_scaling(outputs)  # Apply temperature scaling
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        all_preds, all_labels, all_probs = [], [], []

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                outputs = model.temperature_scaling(outputs)  # Apply temperature scaling
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                probs = F.softmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs)

        val_loss /= len(valid_loader)
        val_acc = correct / total

        # Compute Metrics
        f1 = f1_score(all_labels, all_preds, average='weighted')
        precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
        auc = roc_auc_score(all_labels, all_probs, multi_class="ovr", average="weighted") 

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["valid_loss"].append(val_loss)
        history["valid_acc"].append(val_acc)
        history["f1"].append(f1)
        history["precision"].append(precision)
        history["recall"].append(recall)
        history["auc"].append(auc)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, AUC: {auc:.4f}")

        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered.")
                break

    history["time"] = time.time() - start_time
    return history

In [21]:
# Store Results
results = {}
training_times = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for mh_name, selector in metaheuristics.items():
    selected_features = selector.select_features(train_loader, valid_loader)
    mh_results = {}

    for model_name, model in models.items():
        model.to(device)  # Move model to GPU if available
        print(f"\nTraining {model_name} with {mh_name} feature selection...")
        history = train_and_evaluate(model, optimizers[model_name], train_loader, valid_loader, num_epochs, device)
        mh_results[model_name] = history

    results[mh_name] = mh_results

print("\nTraining Time per Model:")
for name, history in results.items():
    total_time = history[list(history.keys())[0]]["time"]  # Get time from any model in the metaheuristic
    print(f"{name}: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

Selecting features using Novel Tuna Swarm Optimization (NTSO)...
Iteration 1/50 - Best Fitness: 0.0203
Iteration 2/50 - Best Fitness: 0.0194
Iteration 3/50 - Best Fitness: 0.0194
Iteration 4/50 - Best Fitness: 0.0194
Iteration 5/50 - Best Fitness: 0.0143
Iteration 6/50 - Best Fitness: 0.0036
Iteration 7/50 - Best Fitness: 0.0036
Iteration 8/50 - Best Fitness: 0.0036
Iteration 9/50 - Best Fitness: 0.0036
Iteration 10/50 - Best Fitness: 0.0036
Iteration 11/50 - Best Fitness: 0.0036
Iteration 12/50 - Best Fitness: 0.0036
Iteration 13/50 - Best Fitness: 0.0033
Iteration 14/50 - Best Fitness: 0.0033
Iteration 15/50 - Best Fitness: 0.0033
Iteration 16/50 - Best Fitness: 0.0033
Iteration 17/50 - Best Fitness: 0.0033
Iteration 18/50 - Best Fitness: 0.0030
Iteration 19/50 - Best Fitness: 0.0030
Iteration 20/50 - Best Fitness: 0.0030
Iteration 21/50 - Best Fitness: 0.0030
Iteration 22/50 - Best Fitness: 0.0030
Iteration 23/50 - Best Fitness: 0.0030
Iteration 24/50 - Best Fitness: 0.0030
Iteratio

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 - Train Loss: 1.2463, Train Acc: 0.4305, Val Loss: 0.9040, Val Acc: 0.5647, F1: 0.5288, Precision: 0.6831, Recall: 0.5647, AUC: 0.9635
Epoch 2/50 - Train Loss: 0.6235, Train Acc: 0.6382, Val Loss: 0.4748, Val Acc: 0.7155, F1: 0.7109, Precision: 0.7516, Recall: 0.7155, AUC: 0.9813
Epoch 3/50 - Train Loss: 0.4934, Train Acc: 0.7019, Val Loss: 0.4845, Val Acc: 0.7227, F1: 0.7204, Precision: 0.7893, Recall: 0.7227, AUC: 0.9819
Epoch 4/50 - Train Loss: 0.3843, Train Acc: 0.7290, Val Loss: 0.5414, Val Acc: 0.6983, F1: 0.7019, Precision: 0.7815, Recall: 0.6983, AUC: 0.9850
Epoch 5/50 - Train Loss: 0.3531, Train Acc: 0.7800, Val Loss: 0.4825, Val Acc: 0.7213, F1: 0.7108, Precision: 0.7670, Recall: 0.7213, AUC: 0.9837
Epoch 6/50 - Train Loss: 0.1448, Train Acc: 0.8799, Val Loss: 0.2180, Val Acc: 0.8290, F1: 0.8308, Precision: 0.8528, Recall: 0.8290, AUC: 0.9936
Epoch 7/50 - Train Loss: 0.1092, Train Acc: 0.9112, Val Loss: 0.2714, Val Acc: 0.8190, F1: 0.8140, Precision: 0.8675, Recall

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 - Train Loss: 1.1412, Train Acc: 0.4951, Val Loss: 1.6856, Val Acc: 0.5575, F1: 0.5271, Precision: 0.5618, Recall: 0.5575, AUC: 0.9296
Epoch 2/50 - Train Loss: 0.5359, Train Acc: 0.6859, Val Loss: 0.4211, Val Acc: 0.8046, F1: 0.8108, Precision: 0.8379, Recall: 0.8046, AUC: 0.9893
Epoch 3/50 - Train Loss: 0.4161, Train Acc: 0.7410, Val Loss: 0.4082, Val Acc: 0.7845, F1: 0.7891, Precision: 0.8352, Recall: 0.7845, AUC: 0.9821
Epoch 4/50 - Train Loss: 0.2994, Train Acc: 0.8084, Val Loss: 0.4221, Val Acc: 0.7945, F1: 0.7939, Precision: 0.8385, Recall: 0.7945, AUC: 0.9892
Epoch 5/50 - Train Loss: 0.2899, Train Acc: 0.8092, Val Loss: 0.3536, Val Acc: 0.7787, F1: 0.7745, Precision: 0.8172, Recall: 0.7787, AUC: 0.9894
Epoch 6/50 - Train Loss: 0.2464, Train Acc: 0.8335, Val Loss: 0.4111, Val Acc: 0.8003, F1: 0.8051, Precision: 0.8525, Recall: 0.8003, AUC: 0.9904
Epoch 7/50 - Train Loss: 0.2004, Train Acc: 0.8618, Val Loss: 0.2626, Val Acc: 0.8204, F1: 0.8216, Precision: 0.8491, Recall

...

Epoch 1/50 - Train Loss: 0.0076, Train Acc: 0.9914, Val Loss: 0.1627, Val Acc: 0.9095, F1: 0.9108, Precision: 0.9225, Recall: 0.9095, AUC: 0.9958
Epoch 2/50 - Train Loss: 0.0092, Train Acc: 0.9914, Val Loss: 0.1571, Val Acc: 0.9195, F1: 0.9210, Precision: 0.9313, Recall: 0.9195, AUC: 0.9962
Epoch 3/50 - Train Loss: 0.0078, Train Acc: 0.9922, Val Loss: 0.1792, Val Acc: 0.9009, F1: 0.9008, Precision: 0.9127, Recall: 0.9009, AUC: 0.9959
Epoch 4/50 - Train Loss: 0.0122, Train Acc: 0.9885, Val Loss: 0.1481, Val Acc: 0.9095, F1: 0.9099, Precision: 0.9192, Recall: 0.9095, AUC: 0.9965
Epoch 5/50 - Train Loss: 0.0065, Train Acc: 0.9910, Val Loss: 0.1479, Val Acc: 0.9095, F1: 0.9102, Precision: 0.9203, Recall: 0.9095, AUC: 0.9966
Epoch 6/50 - Train Loss: 0.0092, Train Acc: 0.9877, Val Loss: 0.1945, Val Acc: 0.9066, F1: 0.9063, Precision: 0.9184, Recall: 0.9066, AUC: 0.9963
Epoch 7/50 - Train Loss: 0.0124, Train Acc: 0.9914, Val Loss: 0.1704, Val Acc: 0.9023, F1: 0.9037, Precision: 0.9159, Recall

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 - Train Loss: 0.0047, Train Acc: 0.9947, Val Loss: 0.1592, Val Acc: 0.9224, F1: 0.9231, Precision: 0.9311, Recall: 0.9224, AUC: 0.9958
Epoch 2/50 - Train Loss: 0.0043, Train Acc: 0.9938, Val Loss: 0.1582, Val Acc: 0.9253, F1: 0.9257, Precision: 0.9308, Recall: 0.9253, AUC: 0.9961
Epoch 3/50 - Train Loss: 0.0035, Train Acc: 0.9979, Val Loss: 0.1533, Val Acc: 0.9253, F1: 0.9259, Precision: 0.9314, Recall: 0.9253, AUC: 0.9956
Epoch 4/50 - Train Loss: 0.0052, Train Acc: 0.9938, Val Loss: 0.1483, Val Acc: 0.9253, F1: 0.9257, Precision: 0.9307, Recall: 0.9253, AUC: 0.9961
Epoch 5/50 - Train Loss: 0.0047, Train Acc: 0.9951, Val Loss: 0.1431, Val Acc: 0.9296, F1: 0.9302, Precision: 0.9358, Recall: 0.9296, AUC: 0.9962
Epoch 6/50 - Train Loss: 0.0057, Train Acc: 0.9934, Val Loss: 0.1510, Val Acc: 0.9310, F1: 0.9316, Precision: 0.9366, Recall: 0.9310, AUC: 0.9959
Epoch 7/50 - Train Loss: 0.0045, Train Acc: 0.9938, Val Loss: 0.1497, Val Acc: 0.9282, F1: 0.9287, Precision: 0.9333, Recall

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 - Train Loss: 0.0071, Train Acc: 0.9889, Val Loss: 0.1423, Val Acc: 0.9267, F1: 0.9271, Precision: 0.9339, Recall: 0.9267, AUC: 0.9969
Epoch 2/50 - Train Loss: 0.0090, Train Acc: 0.9901, Val Loss: 0.1290, Val Acc: 0.9181, F1: 0.9185, Precision: 0.9239, Recall: 0.9181, AUC: 0.9969
Epoch 3/50 - Train Loss: 0.0061, Train Acc: 0.9910, Val Loss: 0.1439, Val Acc: 0.9210, F1: 0.9216, Precision: 0.9268, Recall: 0.9210, AUC: 0.9967
Epoch 4/50 - Train Loss: 0.0075, Train Acc: 0.9922, Val Loss: 0.1516, Val Acc: 0.9267, F1: 0.9270, Precision: 0.9330, Recall: 0.9267, AUC: 0.9970
Epoch 5/50 - Train Loss: 0.0070, Train Acc: 0.9922, Val Loss: 0.1439, Val Acc: 0.9224, F1: 0.9230, Precision: 0.9293, Recall: 0.9224, AUC: 0.9969
Epoch 6/50 - Train Loss: 0.0054, Train Acc: 0.9938, Val Loss: 0.1405, Val Acc: 0.9167, F1: 0.9172, Precision: 0.9246, Recall: 0.9167, AUC: 0.9970
Epoch 7/50 - Train Loss: 0.0040, Train Acc: 0.9955, Val Loss: 0.1324, Val Acc: 0.9282, F1: 0.9284, Precision: 0.9329, Recall

In [22]:
# Create a folder to save the plots
save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/1. Single Metahurestic with Testing Time/Accuracy_Loss_Curves"
os.makedirs(save_dir, exist_ok=True)

# Function to plot accuracy and loss curves
def plot_metrics(history, model_name, dataset_name):
    epochs = range(1, len(history["train_acc"]) + 1)  # Fix: Use actual history length
    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history["train_acc"], label="Train Accuracy")
    plt.plot(epochs, history["valid_acc"], label="Validation Accuracy")
    plt.title(f"{model_name} ({dataset_name}) Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["valid_loss"], label="Validation Loss")
    plt.title(f"{model_name} ({dataset_name}) Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    # Save the plot with dataset name included
    filename = f"{model_name.replace(' ', '_')}_{dataset_name.replace(' ', '_')}_metrics.png"
    plt.savefig(os.path.join(save_dir, filename))
    plt.close()

# Generate and save accuracy/loss curves for all models
for mh_name, mh_results in results.items():
    for model_name, history in mh_results.items():
        model_key = f"{mh_name}_{model_name}"  # Consistent naming

        # Plot for Original Test Set
        plot_metrics(history, model_key, "Original_Test_Set")

        # Plot for External Test Set
        plot_metrics(history, model_key, "External_Test_Set")

In [23]:
# Function to evaluate model and save classification report
def evaluate_model(model, test_loader, model_name, dataset_name):
    model.eval()
    y_true, y_pred = [], []
    
    start_time = time.time()  # Start timing

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    end_time = time.time()  # End timing
    testing_time = round((end_time - start_time) * 1000, 2)  # Convert to milliseconds

    report = classification_report(y_true, y_pred, target_names=class_names, digits=4, output_dict=True)
    print(f"\nClassification Report for {model_name} on {dataset_name}:\n")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    
    return y_true, y_pred, report, testing_time

    

# Evaluate each model on both test sets and store results
model_reports = {}
predictions = {}
testing_times = {}

for mh_name, mh_results in results.items():  # Iterate over metaheuristics
    for model_name in mh_results.keys():  # Iterate over models in each metaheuristic
        model = models[model_name]  # Retrieve actual model from models dictionary

        # Evaluate on Original Test Set
        y_true, y_pred, report, test_time = evaluate_model(model, test_loader, f"{mh_name}_{model_name}", "Original Test Set")
        model_reports[f"{mh_name}_{model_name}_Test"] = report
        predictions[f"{mh_name}_{model_name}_Test"] = (y_true, y_pred)
        testing_times[f"{mh_name}_{model_name}_Test"] = test_time

        # Evaluate on External Test Set
        y_true, y_pred, report, test_time = evaluate_model(model, external_test_loader, f"{mh_name}_{model_name}", "External Test Set")
        model_reports[f"{mh_name}_{model_name}_External_Test"] = report
        predictions[f"{mh_name}_{model_name}_External_Test"] = (y_true, y_pred)
        testing_times[f"{mh_name}_{model_name}_External_Test"] = test_time


Classification Report for NTSO_ConvNeXt-Tiny on Original Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.8333    0.8333    0.8333         6
    A_blk_spot     0.8182    0.9000    0.8571        10
    A_e_canker     0.8824    0.8824    0.8824        17
  A_gl_lf_spot     0.9333    0.9032    0.9180        31
   A_healthy_f     0.9444    1.0000    0.9714        17
   A_healthy_l     0.8125    0.7647    0.7879        17
     A_m_virus     0.8947    0.8500    0.8718        20
Av_alg_lf_spot     0.8889    1.0000    0.9412         8
  Av_br_canker     0.7143    1.0000    0.8333        25
  Av_healthy_l     0.9286    0.5652    0.7027        23
  G_blk_spot_c     0.8000    1.0000    0.8889         8
   G_healthy_c     1.0000    0.8824    0.9375        17
 Kf_bac_canker     0.9697    1.0000    0.9846        32
  Kf_healthy_l     1.0000    0.9630    0.9811        27
      P_canker     0.8333    1.0000    0.9091        10
   P_fr_blight     1.0000    0.833

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap


Classification Report for NTSO_ConvNeXt-Tiny on External Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.7015    0.7460    0.7231        63
    A_blk_spot     0.2667    0.5333    0.3556        15
    A_e_canker     0.5833    0.5833    0.5833        12
  A_gl_lf_spot     0.8571    0.4000    0.5455        15
   A_healthy_f     0.8333    0.7955    0.8140        44
   A_healthy_l     0.0000    0.0000    0.0000        15
     A_m_virus     0.7273    0.6486    0.6857        37
Av_alg_lf_spot     0.3462    0.8182    0.4865        11
  Av_br_canker     0.3947    0.7895    0.5263        19
  Av_healthy_l     0.3913    0.3000    0.3396        30
  G_blk_spot_c     0.5000    0.2000    0.2857         5
   G_healthy_c     0.9787    0.5412    0.6970        85
 Kf_bac_canker     1.0000    0.4286    0.6000        14
  Kf_healthy_l     1.0000    0.3714    0.5417        35
      P_canker     0.3529    0.5455    0.4286        11
   P_fr_blight     0.3636    0.800

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap


Classification Report for PMOA_ConvNeXt-Tiny on External Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.7015    0.7460    0.7231        63
    A_blk_spot     0.2667    0.5333    0.3556        15
    A_e_canker     0.5833    0.5833    0.5833        12
  A_gl_lf_spot     0.8571    0.4000    0.5455        15
   A_healthy_f     0.8333    0.7955    0.8140        44
   A_healthy_l     0.0000    0.0000    0.0000        15
     A_m_virus     0.7273    0.6486    0.6857        37
Av_alg_lf_spot     0.3462    0.8182    0.4865        11
  Av_br_canker     0.3947    0.7895    0.5263        19
  Av_healthy_l     0.3913    0.3000    0.3396        30
  G_blk_spot_c     0.5000    0.2000    0.2857         5
   G_healthy_c     0.9787    0.5412    0.6970        85
 Kf_bac_canker     1.0000    0.4286    0.6000        14
  Kf_healthy_l     1.0000    0.3714    0.5417        35
      P_canker     0.3529    0.5455    0.4286        11
   P_fr_blight     0.3636    0.800

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap


Classification Report for LOA_ConvNeXt-Tiny on External Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.7015    0.7460    0.7231        63
    A_blk_spot     0.2667    0.5333    0.3556        15
    A_e_canker     0.5833    0.5833    0.5833        12
  A_gl_lf_spot     0.8571    0.4000    0.5455        15
   A_healthy_f     0.8333    0.7955    0.8140        44
   A_healthy_l     0.0000    0.0000    0.0000        15
     A_m_virus     0.7273    0.6486    0.6857        37
Av_alg_lf_spot     0.3462    0.8182    0.4865        11
  Av_br_canker     0.3947    0.7895    0.5263        19
  Av_healthy_l     0.3913    0.3000    0.3396        30
  G_blk_spot_c     0.5000    0.2000    0.2857         5
   G_healthy_c     0.9787    0.5412    0.6970        85
 Kf_bac_canker     1.0000    0.4286    0.6000        14
  Kf_healthy_l     1.0000    0.3714    0.5417        35
      P_canker     0.3529    0.5455    0.4286        11
   P_fr_blight     0.3636    0.8000

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap


Classification Report for OOA_ConvNeXt-Tiny on External Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.7015    0.7460    0.7231        63
    A_blk_spot     0.2667    0.5333    0.3556        15
    A_e_canker     0.5833    0.5833    0.5833        12
  A_gl_lf_spot     0.8571    0.4000    0.5455        15
   A_healthy_f     0.8333    0.7955    0.8140        44
   A_healthy_l     0.0000    0.0000    0.0000        15
     A_m_virus     0.7273    0.6486    0.6857        37
Av_alg_lf_spot     0.3462    0.8182    0.4865        11
  Av_br_canker     0.3947    0.7895    0.5263        19
  Av_healthy_l     0.3913    0.3000    0.3396        30
  G_blk_spot_c     0.5000    0.2000    0.2857         5
   G_healthy_c     0.9787    0.5412    0.6970        85
 Kf_bac_canker     1.0000    0.4286    0.6000        14
  Kf_healthy_l     1.0000    0.3714    0.5417        35
      P_canker     0.3529    0.5455    0.4286        11
   P_fr_blight     0.3636    0.8000

In [24]:
# Create directories
cm_save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/1. Single Metahurestic with Testing Time/Confusion Matrix"
os.makedirs(cm_save_dir, exist_ok=True)

# Function to plot and save confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names, model_name, dataset_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix - {model_name} ({dataset_name})')

    # Save confusion matrix
    cm_filename = f"{model_name.replace(' ', '_')}_{dataset_name}_confusion_matrix.png"
    cm_path = os.path.join(cm_save_dir, cm_filename)
    plt.savefig(cm_path)
    plt.close()

# Generate and save confusion matrices for both test sets
for mh_name, mh_results in results.items():
    for model_name in mh_results.keys():
        model_key = f"{mh_name}_{model_name}"  # Consistent naming

        # Confusion Matrix for Original Test Set
        y_true, y_pred = predictions[f"{model_key}_Test"]
        plot_confusion_matrix(y_true, y_pred, class_names, model_key, "Original_Test_Set")

        # Confusion Matrix for External Test Set
        y_true, y_pred = predictions[f"{model_key}_External_Test"]
        plot_confusion_matrix(y_true, y_pred, class_names, model_key, "External_Test_Set")

In [25]:
# Create a folder to save the ROC curves
roc_save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/1. Single Metahurestic with Testing Time/ROC Curves"
os.makedirs(roc_save_dir, exist_ok=True)

# Function to compute AUC and save ROC plots for both test sets
def compute_auc(model, test_loader, class_names, model_name, dataset_name):
    model.eval()
    y_true, y_scores = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)  # Convert to probabilities

            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probabilities.cpu().numpy())

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    auc_scores = {}
    plt.figure(figsize=(10, 7))

    for i in range(len(class_names)):  # Loop through each class
        fpr, tpr, _ = roc_curve(y_true == i, y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        auc_scores[class_names[i]] = round(roc_auc, 4)  # Store rounded AUC
        plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.4f})")

    plt.plot([0, 1], [0, 1], "k--")  # Diagonal line for reference
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {model_name} ({dataset_name})")
    plt.legend(loc="lower right")

    # Save the ROC curve with dataset name
    filename = f"{model_name.replace(' ', '_')}_{dataset_name.replace(' ', '_')}_roc_curve.png"
    plt.savefig(os.path.join(roc_save_dir, filename))
    plt.close()

    return auc_scores

# Compute and store AUC scores for each model on both test sets
auc_results = {}

for mh_name, mh_results in results.items():
    for model_name in mh_results.keys():
        model_key = f"{mh_name}_{model_name}"  # Consistent naming

        # Compute AUC for Original Test Set
        auc_scores_test = compute_auc(models[model_name], test_loader, class_names, model_key, "Original_Test_Set")
        auc_results[f"{model_key}_Test"] = auc_scores_test

        # Compute AUC for External Test Set
        auc_scores_external = compute_auc(models[model_name], external_test_loader, class_names, model_key, "External_Test_Set")
        auc_results[f"{model_key}_External_Test"] = auc_scores_external

In [26]:
# Extract relevant metrics
model_names = []
dataset_types = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
auc_mean_scores = []
training_times = []
testing_time_values = [] # Extract relevant metrics

for name, report in model_reports.items():
    dataset_type = "External Test Set" if "External_Test" in name else "Original Test Set"
    base_name = name.replace("_Test", "").replace("_External_Test", "")

    model_names.append(base_name)
    dataset_types.append(dataset_type)
    
    accuracy_scores.append(round(report["accuracy"] * 100, 2))  # Convert to percentage
    precision_scores.append(round(report["macro avg"]["precision"] * 100, 2))
    recall_scores.append(round(report["macro avg"]["recall"] * 100, 2))
    f1_scores.append(round(report["macro avg"]["f1-score"] * 100, 2))
    
    # Compute mean AUC across all classes
    mean_auc = np.mean(list(auc_results[name].values()))
    auc_mean_scores.append(round(mean_auc * 100, 2))

    # Extract training time from results
    training_time = results[base_name.split("_")[0]][base_name.split("_")[1]]["time"]
    training_times.append(round(training_time / 60, 2))  # Convert to minutes

    # Extract testing time
    testing_time_values.append(testing_times[name])

# Create DataFrame with Testing Time
df_metrics = pd.DataFrame({
    "Model": model_names,
    "Dataset": dataset_types,
    "Accuracy (%)": accuracy_scores,
    "Precision (%)": precision_scores,
    "Recall (%)": recall_scores,
    "F1-Score (%)": f1_scores,
    "Mean AUC (%)": auc_mean_scores,
    "Training Time (mins)": training_times,
    "Testing Time (ms)": testing_time_values
})

# Display the table
print("\nModel Performance Comparison for Both Test Sets:\n")
print(df_metrics)

# Save DataFrame to CSV
csv_path = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/1. Single Metahurestic with Testing Time/model_performance_metrics_single.csv"
df_metrics.to_csv(csv_path, index=False)


Model Performance Comparison for Both Test Sets:

                           Model            Dataset  Accuracy (%)  \
0             NTSO_ConvNeXt-Tiny  Original Test Set         90.52   
1    NTSO_ConvNeXt-Tiny_External  External Test Set         60.19   
2            NTSO_EfficientNetV2  Original Test Set         91.09   
3   NTSO_EfficientNetV2_External  External Test Set         72.88   
4               NTSO_RegNetY-8GF  Original Test Set         93.39   
5      NTSO_RegNetY-8GF_External  External Test Set         66.73   
6                    NTSO_MaxViT  Original Test Set         93.10   
7           NTSO_MaxViT_External  External Test Set         70.96   
8             PMOA_ConvNeXt-Tiny  Original Test Set         90.52   
9    PMOA_ConvNeXt-Tiny_External  External Test Set         60.19   
10           PMOA_EfficientNetV2  Original Test Set         91.09   
11  PMOA_EfficientNetV2_External  External Test Set         72.88   
12              PMOA_RegNetY-8GF  Original Test Set 